In [1]:
import numpy as np
import json
import re
from settings import settings
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import PyPDF2
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize


def remove_punctuation(text):
    punctuation_pattern = r"[^\w\s]"
    return re.sub(punctuation_pattern, "", text)


def extract_tokens_from_regular_input(query):
    return remove_punctuation(query.lower()).strip().split()


def extract_tokens_from_file_input(file):
    pdf_read = PyPDF2.PdfReader(file)
    resume_text = ""
    for page in pdf_read.pages:
        resume_text += page.extract_text() or ""
    return extract_tokens_from_regular_input(remove_punctuation(resume_text))


def extract_tokens_from_docs(doc):
    attrs = [doc["company"], doc["role"], doc["country"], doc["city"], doc["skills"]]
    return remove_punctuation(" ".join(attrs).lower())


def tokenize_docs(doc):
    return doc.split()


def construct_invertex_index(vectorizer, tfidf_matrix, n_components):
    feature_names = vectorizer.get_feature_names_out()
    print(feature_names)
    terms_indices = {
        term: idx for idx, term in enumerate(feature_names) if idx < n_components
    }
    inverted_index = defaultdict(list)
    rows, cols = tfidf_matrix.nonzero()
    for row, col in zip(rows, cols):
        # if col < n_components:  # Only include columns within the reduced dimensions
        inverted_index[feature_names[col]].append((row, tfidf_matrix[row, col]))
    return terms_indices, inverted_index


def construct_term_idf_map(vectorizer):
    return dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))


def construct_query_tfidf(query, idf_map):
    tokens = Counter(extract_tokens_from_regular_input(query))
    return {
        term: freq * idf_map.get(term, 0)
        for term, freq in tokens.items()
        if term in idf_map
    }


def compute_cosine_scores(query):
    query_tfidf = vectorizer.transform([query]).toarray()
    query_vec = normalize(np.dot(query_tfidf, words_compressed)).squeeze()
    sims = docs_compressed_normed.dot(query_vec)
    asort = np.argsort(-sims)[:200]
    return [(i, sims[i]) for i in asort[1:]]



def compute_greatest_sim(query):
    query_tfidf = vectorizer.transform([query]).toarray()
    query_vec = normalize(np.dot(query_tfidf, words_compressed)).squeeze()
    sims = docs_compressed_normed.multiply(query_vec)

def closest_projects_to_word(word_in, k=5):
    if word_in not in word_to_index:
        return []
    sims = docs_compressed_normed.dot(
        words_compressed_normed[word_to_index[word_in], :]
    )
    asort = np.argsort(-sims)[: k + 1]
    return [(i, documents[i][0], sims[i]) for i in asort[1:]]


# Load data and prepare TF-IDF and SVD
with open(settings.data_file_path, "r") as file:
    data = json.load(file)
documents = data.get("job_postings")

vectorizer = TfidfVectorizer(tokenizer=tokenize_docs, stop_words="english")

td_matrix = vectorizer.fit_transform(
    [extract_tokens_from_docs(doc) for doc in documents]
)

docs_compressed, s, words_compressed = svds(td_matrix, k=300)
words_compressed = words_compressed.transpose()

word_to_index = vectorizer.vocabulary_
index_to_word = {i: t for t, i in word_to_index.items()}

words_compressed_normed = normalize(words_compressed, axis=1)


td_matrix_np = td_matrix.transpose().toarray()
td_matrix_np = normalize(td_matrix_np)


docs_compressed_normed = normalize(docs_compressed)


In [2]:
from index import documents, compute_cosine_scores, extract_tokens_from_file_input


def group_postings_by_company(postings):
    groups = {}

    for p in postings:
        company = p["company"]

        if company in groups:
            groups[company]["postings"].append(p)
        else:
            groups[company] = {
                "name": company,
                "description": p["description"],
                "rating": p["rating"],
                "happiness": p["happiness"],
                "postings": [p],
            }

    return groups


def select_similar_k_docs(doc_scores_map, k=25):
    res = []

    for doc, score in doc_scores_map:
        document = documents[doc]
        document["score"] = score
        res.append(document)

        if k == 0:
            break

    return res


def get_postings_regular_input(text, k=25):
    cosine_scores = compute_cosine_scores(text)
    return select_similar_k_docs(cosine_scores, k)


def get_postings_file_input(file, k=25):
    file_tokens = extract_tokens_from_file_input(file)
    cosine_scores = compute_cosine_scores(" ".join(file_tokens))
    return select_similar_k_docs(cosine_scores, k)


In [3]:
#query = "Proficiency in one or more backend programming languages (e.g., Java, Python, Node.js, Ruby) API development Database management (SQL or NoSQL) Server management and deployment Security best practices Knowledge of web frameworks (e.g., Express, Django)"

In [4]:
# query = "Highly skilled and compassionate Registered Nurse with over 7 years of experience in various healthcare settings. Proficient in providing comprehensive patient care, including assessment, planning, and implementation of nursing interventions. Strong ability to communicate effectively with patients, families, and interdisciplinary teams to ensure optimal patient outcomes. Experienced in managing diverse patient populations, ranging from pediatrics to geriatrics, with a focus on delivering evidence-based care. Proven track record of excellence in clinical skills, including medication administration, wound care, and patient education. Dedicated to continuous professional development and staying abreast of the latest advancements in nursing practice. Seeking to leverage my expertise and passion for patient care in a dynamic healthcare environment."

In [56]:
query = "hi"

In [57]:
#Start with a query
query

'hi'

In [58]:
#Function to allow us to compare each individual job with the query by creating large array of repeat queries
def repeat_vector(vector, length):
    # Reshape the vector to be a row vector
    vector = np.array(vector).reshape(1, -1)
    
    # Use broadcasting to repeat the vector
    repeated_array = np.tile(vector, (length, 1))
    
    return repeated_array

In [59]:
#Create TFidf
query_tfidf = vectorizer.transform([query]).toarray()
query_vec = normalize(np.dot(query_tfidf, words_compressed)).squeeze()

#Create repeated query vector to analyze how much each index contributes in the dot product (cosine score)
length = len(docs_compressed_normed)
repeated_query_vec = repeat_vector(query_vec, length)

#Multiply by each document
sims = docs_compressed_normed *repeated_query_vec

#Extract out the indices that contribute the most to the cosine score and their values
largest_indices = np.argsort(-sims, axis=1)
largest_values = np.sort(-sims, axis=1)


In [60]:
#Now, extract the relation between the SVD terms and our query
#Extract all the words in the tokenizer
words = vectorizer.get_feature_names_out()
#Find out which value in our query is most related to each SVD index
max_indices = np.argmax(np.expand_dims(query_tfidf.squeeze(), axis=-1) * words_compressed_normed, axis=0)
#This creates our dictionary to display our statistics
word_representations = []
for i in max_indices:
    word_representations.append(words[i])

In [61]:
#Now calculuate the words that link the query to each job posting, the most similar words between them
most_sim_words = []
sim_scores = []
k = 5 #top k keys

n,m = largest_indices.shape
for i in range(n):
    #Track contribution of each word in their similarity score
    contribution_per_index = {word: 0 for word in word_representations}
    for j in range(m):
         contribution_per_index[word_representations[largest_indices[i,j]]] -= largest_values[i,j]   
    print(contribution_per_index)
    #Find top k related words
    top_k_words = sorted(contribution_per_index, key=contribution_per_index.get, reverse=True)[:k]
    
    #Track all the scores, this will show the percentage the top k related words contribute to the cosine score
    sorted_values = sorted(contribution_per_index.values(), reverse=True)
    
    #Track
    most_sim_words.append(top_k_words)
    sim_scores.append(sorted_values)

{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}

{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}

{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}
{'1': 0.0}


In [62]:
#Calculuate similarity scores
sims = docs_compressed_normed.dot(query_vec)
p = 10
#Find the top p postings
asort = np.argsort(-sims)[:p]
#Get their most similar words
result = [most_sim_words[i] for i in asort]
#Get the contribution of each word
result_scores = [sim_scores[i] for i in asort]

In [63]:
def preprocess_results(array1, array2):
    #removes 1s that results from no contribution of words
    new_array1 = []
    new_array2 = []

    # Iterate over pairs of sub-arrays from both input arrays
    for sub_array1, sub_array2 in zip(array1, array2):
        # Use list comprehension to filter out the '1's and their corresponding elements
        filtered_sub_array1 = [elem for elem in sub_array1 if elem != '1']
        filtered_sub_array2 = [elem for elem, orig_elem in zip(sub_array2, sub_array1) if orig_elem != '1']

        # Append the filtered sub-arrays to the new arrays
        new_array1.append(filtered_sub_array1)
        new_array2.append(filtered_sub_array2)

    return new_array1, new_array2

In [64]:
result

[['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1']]

In [65]:
result_scores

[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]

In [66]:
result, result_scores = remove_ones_and_corresponding(result, result_scores)

In [67]:
# Find the sum of total contributions for each word
sums = np.sum(np.abs(result_scores), axis=1)

# Calculate the percentage contribution of each word
percentages = (result_scores / sums[:, np.newaxis] + 0.0000001) * 100


In [68]:
#Display top k
percentages = percentages[:,:k]
percentages

array([], shape=(10, 0), dtype=float64)

In [69]:
result

[[], [], [], [], [], [], [], [], [], []]

In [70]:
#Now graph using a spider plot
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [71]:
data = {
    'word': result[0],
    'percent': percentages[0,0:5]
}
df = pd.DataFrame(data)

In [72]:
layout = go.Layout(title='Contribution Percentages of Top 5 Words in Your Resume')

fig = go.Figure(layout=layout)
fig.add_trace(
    go.Scatterpolar(
        r=df['percent'],
        theta=df['word'],
        fill='toself',
        fillcolor='lightgreen',
        name='Contribution Percentages'
    )
)


fig.show()